In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib
from flask import Flask, request, jsonify
import google.generativeai as genai
import os

from google.colab import drive
drive.mount('/content/drive')


# API key
os.environ['GOOGLE_API_KEY'] = "090078601huzaifa"

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
model = genai.GenerativeModel('gemini-pro')

# Load and preprocess dataset
df = pd.read_csv('/content/drive/My Drive/kidney_disease.csv')  # Update the path to your dataset

# Preprocessing steps (Label Encoding and handling missing values)
lb = LabelEncoder()
df['rbc'] = lb.fit_transform(df['rbc'])
df['pc'] = lb.fit_transform(df['pc'])
df['pcc'] = lb.fit_transform(df['pcc'])
df['ba'] = lb.fit_transform(df['ba'])
df['htn'] = lb.fit_transform(df['htn'])
df['dm'] = lb.fit_transform(df['dm'])
df['cad'] = lb.fit_transform(df['cad'])
df['appet'] = lb.fit_transform(df['appet'])
df['pe'] = lb.fit_transform(df['pe'])
df['ane'] = lb.fit_transform(df['ane'])
df['classification'] = lb.fit_transform(df['classification'])

# Handle missing data and convert columns to numeric
df['age'] = df['age'].fillna(df['age'].mean())  # Fill missing age with mean
df.replace('\t?', float('nan'), inplace=True)  # Replace '\t?' with NaN

columns_to_convert = ['bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu', 'sc', 'sod', 'pot',
                      'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'classification']
for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

df.dropna(inplace=True)  # Drop rows with missing values

# Prepare features and target variable
X = df.drop(columns=['classification'])
y = df['classification']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Train the KNN model
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)

# Save the trained model using joblib (so it doesn't need to be retrained every time)
joblib.dump(knn, 'knn_model.pkl')


# Create Flask app
app = Flask(__name__)


def generate_gemini_response(prompt):
    """Generates a response using Gemini Pro."""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error generating chatbot response: {e}"


# Prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    # Get input data from the client (C# app)
    data = request.get_json(force=True)
    input_data = np.array(data['input_data']).reshape(1, -1)  # Reshape for single prediction

    # Load the trained model (you could also load the model outside the function to optimize)
    knn_model = joblib.load('knn_model.pkl')

    # Make prediction
    prediction = knn_model.predict(input_data)

    # Return the prediction as a JSON response
    return jsonify({'prediction': prediction.tolist()})


# Chatbot endpoint
@app.route('/chat', methods=['POST'])
def chat():
    # Get user message from the client
    data = request.get_json(force=True)
    user_message = data.get('message', '') # Get message from JSON or use a default

    if not user_message:
        return jsonify({"response": "Please provide a message."})

    # Create the prompt for Gemini. You may want to inject context here (like the prediction result)
    prompt = (f"You are an AI assistant that gives health information about kidney diseases. "
            f"The patient message is: {user_message}. "
            "Please respond with information about kidney diseases"
            )

    # Generate response from Gemini model
    gemini_response = generate_gemini_response(prompt)

    # Respond to the user with the chatbot response
    return jsonify({"response": gemini_response})


if __name__ == '__main__':
    app.run(debug=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
